In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics as sk_metrics
import sys

In [2]:
sys.path.append('../..')
import src.metrics.binary_classification as bin_metrics

In [3]:
path = '../../static/datasets/modified/'
folders = {'bin' : 'bin_classification/', 'multi' : 'multi_classification/'}

train = 'bayes_train.csv'
test = 'bayes_test.csv'

In [10]:
X_train = pd.read_csv(path + folders['bin'] + train)
y_train = pd.read_csv(path + folders['bin'] + test)
X_train

,version https://git-lfs.github.com/spec/v1
0,oid sha256:4b044712609866bc97957a6b4c97edcd3b7...
1,size 21491832


In [5]:
from src.models.baselines import NaiveBayesClassifier as NB_clf
from src.models.baselines import DummyClassifier as Dummy_clf

In [6]:
import numpy as np
import pandas as pd
import scipy
from math import log, exp, sqrt, pi
from random import choices
from abc import ABC, abstractmethod
from tqdm import tqdm

class NaiveBayesClassifier:
    class Distribution:
        def __init__(self, feature_column, mode: str):
            self.mode = mode
            if mode == 'gaussian_kde':
                self.distribution = scipy.stats.gaussian_kde(feature_column)
            if mode == 'gaussian':
                self.mean = feature_column.mean()
                self.deviation = np.std(feature_column)
            if mode == 'cat_features':
                unique, counts = np.unique(feature_column, return_counts=True)
                self.distribution = dict(zip(unique, counts / feature_column.size)) 
        
        def get_proba(self, value: float) -> float:
            if self.mode == 'gaussian':
                return self.get_prob_from_gauss(self, value, self.mean, self.deviation)
            if self.mode == 'gaussian_kde':
                return self.distribution.pdf(value)
            return self.distribution[value]
        
        @staticmethod
        def get_prob_from_gauss(self, value: float, mean: float, deviation: float) -> float:
            if deviation == 0:
                return 1
            return exp(-((value - mean) ** 2) / (2 * deviation ** 2)) / sqrt(2 * pi * deviation ** 2)
    
    def __init__(self, mode = 'gaussian_kde'): #guassian_kde, guassian, cat_feautures
        self.mode = mode
    
    
    def fit(self, train_feature_matrix, train_labels):
        self.unique_labels = np.unique(train_labels)
        self.class_probability = {} # здесь будем хранить долю каждого класса в выборке
        self.distributions = {} # key: label, value: []
                                # [i] - плотность распределения i-й фичи в X при условии, что класс = label, т.е. плотность распределения P(x_i | label))
        for label in self.unique_labels:
            current_label_feature_matrix = train_feature_matrix[np.array(train_labels == label)] # выбираем те строки, для которых класс = label
            self.class_probability[label] = current_label_feature_matrix.size / train_feature_matrix.size
            self.distributions[label] = []
            for feature_ind in range(train_feature_matrix.shape[1]):
                feature_column = current_label_feature_matrix.iloc[:, feature_ind] # выбираем фичу с номером feature_ind
                distribution = self.Distribution(feature_column, self.mode)  # считаем плотность распределения P(x_feature_ind | label) с помощью gaussian_kde
                self.distributions[label].append(distribution)
    
    def predict(self, test_feature_matrix):
        y_pred = [] # наши предсказания
        for i, features in test_feature_matrix.iterrows():
            max_likelihood = -float('inf')
            for label in self.unique_labels: # перебираем возможные варианты ответа, выбираем - максимально правдоподобный
                likelihood = log(self.class_probability[label]) # здесь - сумма логарифмов вероятностей для label
                for feature_ind in range(test_feature_matrix.shape[1]):
                    likelihood += log(self.distributions[label][feature_ind].get_proba(float(features[feature_ind])))
                if likelihood > max_likelihood:
                    max_likelihood = likelihood
                    predict = label
            y_pred.append(predict)
        return y_pred

In [7]:
nb_clf = NaiveBayesClassifier(mode='cat_features')
#dummy_clf = Dummy_clf()

In [8]:

nb_clf.fit(X_train, y_train)
#dummy_clf.fit(X_train, y_train)

In [9]:
y_pred_nb = nb_clf.predict(X_train)
#y_pred_dummy = dummy_clf.predict(X_train)

ValueError: could not convert string to float: 'oid sha256:4b044712609866bc97957a6b4c97edcd3b70c5371c14bbbfd57aaf2f41003cdf'

In [138]:
y_pred_nb = list(map(int, y_pred_nb))
#y_pred_dummy = list(map(int, y_pred_dummy))

In [139]:
sk_metrics.accuracy_score(y_true=y_train, y_pred=y_pred_nb)
#bin_metrics.accuracy_score(y_true=y_train, y_pred=y_pred_dummy)

0.5405260332796564

In [ ]:
# %env CLEARML_WEB_HOST=https://app.clear.ml
# %env CLEARML_API_HOST=https://api.clear.ml
# %env CLEARML_FILES_HOST=https://files.clear.ml
# %env CLEARML_API_ACCESS_KEY=VT1X7EH5UU1REKI7T44X
# %env CLEARML_API_SECRET_KEY=NHZDqveny2UDyuTvhVADxwUOAgp2mVv3wyipHnKkCc42hiaJog

In [ ]:
# from clearml import Task, Logger

In [ ]:
# task = Task.init(
#     project_name='tonality analysis',
#     task_name='TestBaselines',
#     tags=['NaiveBayesClassifier', 'DummyClassifier'])

In [ ]:
# task.upload_artifact(name='train_data_preprocessed_for_NB', artifact_object='../../../static/datasets/modified/bin_classification/bayes_train.csv')
# task.upload_artifact(name='test_data_preprocessed_for_NB', artifact_object='../../../static/datasets/modified/bin_classification/bayes_test.csv')

In [ ]:
# task.close()

In [3]:
from text_analyzer.src.models.baselines import MulticlassClassifier
from text_analyzer.src.metrics import multiclass_classification as multi_metrics

In [5]:
data = pd.read_csv(path + folders['multi'] + 'train_data.csv')
data.dropna(inplace=True)

In [4]:
# X_train = data.iloc[:, :-1]
# y_train = data['label'].astype(int)
X_train = pd.read_csv(path + folders['multi'] + train)
y_train = pd.read_csv(path + folders['multi'] + test)

In [5]:
#from sklearn.naive_bayes import BernoulliNB
#from sklearn.dummy import DummyClassifier

In [6]:
strategies = ["one-vs-all", "all-vs-all"]
for strategy in strategies:
    multi_clf = MulticlassClassifier(BernoulliNB, strategy)
    multi_clf.fit(X_train, y_train, target_name='label')
    y_pred = multi_clf.predict(X_train)
    print(sk_metrics.accuracy_score(y_true=y_train, y_pred=y_pred), sk_metrics.recall_score(y_true=y_train, y_pred=y_pred, average='macro'), sk_metrics.precision_score(y_true=y_train, y_pred=y_pred, average='macro'))

0.7434428700633102 0.42216983521360846 0.6706397937883785
0.7362074163400664 0.38396145025244266 0.7689475778385205
